In [1]:
!pip install --upgrade transformers tensorflow pandas s3fs

     |████████████████████████████████| 675kB 17.9MB/s eta 0:00:01
     |██████████████████████▍         | 361.8MB 92.2MB/s eta 0:00:023   |█▉                              | 28.9MB 71.3MB/s eta 0:00:07     |████████████▋                   | 202.6MB 102.3MB/s eta 0:00:04

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 516.2MB 21kB/s s eta 0:00:01
     |████████████████████████████████| 10.1MB 54.1MB/s eta 0:00:01
     |████████████████████████████████| 3.8MB 54.2MB/s eta 0:00:01
     |████████████████████████████████| 71kB 17.8MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 48.3MB/s eta 0:00:01��▎                        | 245kB 48.3MB/s eta 0:00:01
     |████████████████████████████████| 665kB 74.0MB/s eta 0:00:01
     |████████████████████████████████| 890kB 80.6MB/s eta 0:00:01
     |████████████████████████████████| 3.0MB 60.0MB/s eta 0:00:01
     |████████████████████████████████| 2.9MB 54.3MB/s eta 0:00:01
     |████████████████████████████████| 460kB 87.0MB/s eta 0:00:01
     |████████████████████████████████| 81kB 19.7MB/s eta 0:00:01
     |████████████████████████████████| 778kB 55.3MB/s eta 0:00:01
     |████████████████████████████████| 92kB 21.7MB/s eta 0:00:01
     |████████████████████████████████| 163kB 82.6MB/s eta 0:00:01
     |█

In [35]:
import sagemaker

In [36]:
sagemaker.s3.S3Downloader.download(local_path="./model/", s3_uri="s3://sagemaker-us-east-1-652081700929/final-distilbert-regression-800k-2020-06-15-07-04-51-437/output/model.tar.gz")

In [37]:
%%sh
cd model
tar -xf model.tar.gz

In [38]:
import pandas as pd
from transformers import AutoTokenizer, AutoConfig, TFAutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased', use_fast=True)
model = TFAutoModelForSequenceClassification.from_pretrained("model")

In [39]:
def predict(text):
    return model(tokenizer.encode(text, return_tensors='tf', max_length=512))[0].numpy()[0][0]

In [23]:
reviews = pd.read_csv("s3://yelp-dataset-pt-9/data/csv/reviews.csv")
businesses = pd.read_json("s3://yelp-dataset-pt-9/data/json/business.json", orient="records", lines=True)

In [24]:
businesses.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,1,None,"Pets, Pet Services, Pet Groomers",None
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,0,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Hardware Stores, Home Services, Building Suppl...","{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ..."
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."


In [ ]:
reviews = pd.read_csv("s3://yelp-dataset-pt-9/data/csv/reviews.csv")

In [5]:
reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,xQY8N_XvtGbearJ5X4QryQ,OwjRMXRC0KyPrIlcjaXeFQ,-MhfebM0QIsKt87iDN-FNw,2.0,5.0,0.0,0.0,"As someone who has worked with many museums, I...",2015-04-15 05:21:16
1,UmFMZ8PyXZTY2QcwzsfQYA,nIJD_7ZXHq-FX8byPMOkMQ,lbrU8StCq3yDfr-QMnGrmQ,1.0,1.0,1.0,0.0,I am actually horrified this place is still in...,2013-12-07 03:16:52
2,LG2ZaYiOgpr2DK_90pYjNw,V34qejxNsCbcgD8C0HVk-Q,HQl28KMwrEKHqhFrrDqVNQ,5.0,1.0,0.0,0.0,I love Deagan's. I do. I really do. The atmosp...,2015-12-05 03:18:11
3,i6g_oA9Yf9Y31qt0wibXpw,ofKDkJKXSKZXu5xJNGiiBQ,5JxlZaqCnk1MnbgRirs40Q,1.0,0.0,0.0,0.0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",2011-05-27 05:30:52
4,6TdNDKywdbjoTkizeMce8A,UgMW8bLE0QMJDCkQ1Ax5Mg,IS4cv902ykd8wj1TR0N3-A,4.0,0.0,0.0,0.0,"Oh happy day, finally have a Canes near my cas...",2017-01-14 21:56:57


In [6]:
reviews.shape

(8021124, 9)

In [17]:
business_counts = reviews['business_id'].value_counts()

In [19]:
business_counts.sort_values(ascending=False).head()

RESDUcs7fIiihp38-d6_6g    10417
4JNXUYY8wbaaDmk3BPzlWw     9536
K7lWdNUhCbcnEvI0NhGewg     7594
f4x1YBxkLrZg652xt2KR5g     6859
cYwJA2A6I12KNkm2rtXd5g     5586
Name: business_id, dtype: int64

In [20]:
random_business = reviews[reviews['business_id'] == 'cYwJA2A6I12KNkm2rtXd5g']

In [21]:
random_business.shape

(5586, 9)

In [22]:
random_business.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
7206912,hJrtkwflyh9iD_EjZhSlng,jMpHxQnZl4I7m3HZVJVpgQ,cYwJA2A6I12KNkm2rtXd5g,4.0,0.0,0.0,0.0,Have been here twice now and both times it was...,2013-09-04 00:24:27
7207150,hclGbDgHjSz2WjgKdd6CGQ,q8BT_2-vBJNqm8NiW-hpeA,cYwJA2A6I12KNkm2rtXd5g,5.0,0.0,0.0,0.0,Absolutely wonderful food. Best burger with av...,2015-10-07 22:51:41
7207212,uwAyexoEmapukI0h5BTsJw,TeSXdvgDlVQkuOII1lAxmg,cYwJA2A6I12KNkm2rtXd5g,5.0,0.0,0.0,0.0,Not one for long reviews. This place rocks. On...,2014-01-17 02:36:21
7207336,IbLgz6CamDtuN1OC-_o-Hg,N6STM9kxnOtxJFYtMLVmLQ,cYwJA2A6I12KNkm2rtXd5g,3.0,0.0,0.0,0.0,Good for the price. \nFarm burger: bun very go...,2014-10-11 05:07:55
7207414,WDrbAVMYtrGbv--M-4drgA,Rk2OyIscXuYb_E19MxEj0Q,cYwJA2A6I12KNkm2rtXd5g,5.0,0.0,0.0,0.0,The food was delicious. The service was excel...,2016-07-25 06:05:55


In [25]:
random_business_avg_rating = random_business['stars'].mean()

In [26]:
random_business_med_rating = random_business['stars'].median()

In [27]:
random_business_avg_rating

3.8947368421052633

In [28]:
random_business_med_rating

4.0

In [29]:
random_business_info = businesses[businesses['business_id'] == 'cYwJA2A6I12KNkm2rtXd5g']

In [30]:
random_business_info

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
201975,cYwJA2A6I12KNkm2rtXd5g,Gordon Ramsay BurGR,3667 Las Vegas Blvd S,Las Vegas,NV,89109,36.110724,-115.172169,4.0,5494,0,"{'Caters': 'False', 'RestaurantsDelivery': 'Fa...","Burgers, Restaurants, American (Traditional)","{'Monday': '11:0-0:0', 'Tuesday': '11:0-0:0', ..."


In [31]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [40]:
preds = random_business['text'].progress_apply(predict)

In [42]:
preds.mean()

3.8947585865753847

In [43]:
business_counts[business_counts < 100]

zWno83CCEC1cPGZvxQPf-g    99
lVKCBBt2wvsL0ST0r6nspg    99
F6c3D1o9Z4Tl6cDorb3WgA    99
j7HO1YeMQGYo3KibMXZ5vg    99
-uYAkAplo7a4WM7c6lDrWQ    99
                          ..
v4oFhyARHh9PE3Q2yKuXrQ     3
osi1-hmnOGsSts7NqoYiOg     3
n8FASgUx6wi3rRC9_wvEsA     3
9wVhKKGxdj1-Jhclr0GqNw     3
IEDYk_9EEfQz4nhp0WtBuQ     3
Name: business_id, Length: 192679, dtype: int64

In [44]:
random_business = reviews[reviews['business_id'] == 'zWno83CCEC1cPGZvxQPf-g']

In [45]:
random_business_avg_rating = random_business['stars'].mean()

In [46]:
random_business_med_rating = random_business['stars'].median()

In [47]:
random_business_avg_rating, random_business_med_rating

(4.0, 5.0)

In [48]:
random_business_info = businesses[businesses['business_id'] == 'zWno83CCEC1cPGZvxQPf-g']

In [49]:
random_business_info

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
13723,zWno83CCEC1cPGZvxQPf-g,Costco Wholesale,3411 St Rose Pkwy,Henderson,NV,89052,35.98737,-115.138768,4.0,92,1,"{'DogsAllowed': 'False', 'ByAppointmentOnly': ...","Shopping, Tires, Gas Stations, Automotive, Who...","{'Monday': '10:0-20:30', 'Tuesday': '10:0-20:3..."


In [50]:
preds = random_business['text'].progress_apply(predict)

In [51]:
preds.mean()

3.959969649411211

In [52]:
il = businesses[businesses['state'] == "IL"]

In [53]:
il.shape

(2034, 14)

In [54]:
il['city'].unique()

array(['Champaign', 'Urbana', 'Monticello', 'Tolono', 'Urbana, Illinois',
       'Rantoul', 'Savoy', 'Saint Joseph', 'Mahomet', 'Villa Grove',
       'Ogden', 'Philo', 'Homer', 'Sidney', 'Tuscola', 'Thomasboro',
       'Joliet', 'St Joseph', 'Penfield', 'Fisher', 'Mansfield',
       'Gifford', 'Broadlands', 'Chicago', 'St. Joseph', 'Schaumburg',
       'Bridgeview', 'Dewey', 'Fithian'], dtype=object)

In [55]:
cu = il[(il['city'] == 'Champaign') | (il['city'] == 'Urbana')]

In [56]:
cu

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
8,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,404 E Green St,Champaign,IL,61820,40.110446,-88.233073,4.5,5,1,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...","Ethnic Food, Food Trucks, Specialty Food, Impo...","{'Monday': '11:30-14:30', 'Tuesday': '11:30-14..."
88,UiyjUa8CQmygujhe-ZYlsw,Movers 4 Less Inc,,Champaign,IL,61822,40.127030,-88.241699,3.0,15,1,"{'BusinessAcceptsCreditCards': 'True', 'Busine...","Home Services, Movers","{'Monday': '8:0-20:0', 'Tuesday': '8:0-20:0', ..."
93,efdPLo00sOD-1Wf77hg5jQ,TruGreen Lawn Care,108 Petry Ct,Champaign,IL,61822,40.137287,-88.238436,2.0,13,1,{'BusinessAcceptsCreditCards': 'True'},"Shopping, Nurseries & Gardening, Tree Services...","{'Monday': '8:0-19:0', 'Tuesday': '8:0-19:0', ..."
265,-LfTBo0oa_uD454ScEW2XA,Merry Ann's Diner,1 E Main St,Champaign,IL,61820,40.118133,-88.242900,3.0,47,0,"{'RestaurantsDelivery': 'True', 'WiFi': 'u'fre...","Restaurants, Diners, Sandwiches, Breakfast & B...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
361,1QG4JRQhm1pZQSPu7Axcog,Tatman's Towing,810 E Perkins Rd,Urbana,IL,61802,40.127897,-88.197377,3.5,6,1,{'BusinessAcceptsCreditCards': 'True'},"Auto Repair, Home Services, Keys & Locksmiths,...","{'Monday': '0:0-23:59', 'Tuesday': '0:0-23:59'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209106,yRKrMWQxQaMeR18s5ajcYw,Arby's,1913 W Springfield Ave,Champaign,IL,61821,40.112422,-88.279457,2.0,7,1,"{'RestaurantsTakeOut': 'True', 'WiFi': ''no'',...","Sandwiches, Fast Food, Restaurants","{'Monday': '10:0-23:0', 'Tuesday': '10:0-23:0'..."
209147,HjCdJCPnxNtVY6wKbaB0PA,Champaign County Clerk,1776 East Washington St,Urbana,IL,61802,40.106404,-88.186691,4.0,4,1,None,Public Services & Government,None
209249,n8SUVgkC_hOaZ7FiJd7sHw,April's Florist,512 E John St,Champaign,IL,61820,40.109259,-88.230699,4.5,10,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Event Planning & Services, Wedding Planning, S...","{'Monday': '9:0-17:30', 'Tuesday': '9:0-17:30'..."
209353,jA-sEoLY9tqzwX7-wv09pA,Studio 9 Hair Design,701 Devonshire Dr,Champaign,IL,61820,40.088439,-88.254522,3.5,4,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Hotels & Travel, Beauty & Spas, Health Retreat...","{'Tuesday': '9:0-20:0', 'Wednesday': '9:0-20:0..."


In [57]:
smoke_shack = 'kCIvm-8hNfD-qrCxiWLsSQ'

In [60]:
random_business = reviews[reviews['business_id'] == smoke_shack]

In [61]:
preds = random_business['text'].progress_apply(predict)

In [62]:
preds.mean()

2.1243374824523924